 Import statements

In [16]:
import cv2
from pytesseract import image_to_string
from PIL import Image
import numpy as np

This bit separates the crossword itself from the screenshot as grid.png

In [8]:
#Where is our image?
image_path = 'Screenshot.png'

#Preparing for cropping
im = cv2.imread(image_path)
im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
im = cv2.GaussianBlur(im, (5, 5), 0)
im = cv2.adaptiveThreshold(im, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 9, 4)
cv2.imwrite("preprocess.jpg", im)

# Grid is the largest contour
contours, _ = cv2.findContours(im, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
max_area = -1
max_idx = -1
for i, contour in enumerate(contours):
    area = cv2.contourArea(contour)
    if max_area < area < (im.shape[0] * im.shape[1] / 2):
        max_area = area
        max_idx = i

#Saving grid.png
rect = cv2.boundingRect(contours[max_idx])
print(rect)
Image.open(image_path).crop((rect[0], rect[1], rect[0] + rect[2], rect[1] + rect[3])).save("grid.png")


(32, 25, 730, 735)


Crossword hint extraction

In [ ]:
def extract_hints_from_image(image_path):
    # Load the image
    image = cv2.imread(image_path)
    
    # Convert the image to grayscale for OCR
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Use pytesseract to extract text from the image
    extracted_text = str(image_to_string(gray))
    
    # Only consider text after "Prize crossword"
    if "Prize crossword" in extracted_text:
        extracted_text = extracted_text.split("Prize crossword")[1].strip()
    
    # Split the text based on 'Across' and 'Down' to separate the hints
    across_hints, down_hints = extracted_text.split("DOWN")
    across_hints = across_hints.replace("ACROSS", "").strip()

    return across_hints, down_hints

extract_hints_from_image(image_path)

This bit finds the green squares and gets coordinates for them

In [15]:
crossword_path = 'grid.png'

def detect_green_squares_and_overlay(crossword_path):
    # Load the image
    image = cv2.imread(crossword_path)
    
    # Convert to HSV for color-based detection
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define range for green color
    lower_green = (40, 25, 25)
    upper_green = (45, 255,255)

    
    # Threshold the HSV image to get only green colors
    mask = cv2.inRange(hsv, lower_green, upper_green)
    
    # Convert the binary mask to a 3-channel image
    mask_colored = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    
    # Overlay the mask on the original image
    alpha = 0.5  # Define the opacity for overlay
    overlay = cv2.addWeighted(image, 1 - alpha, mask_colored, alpha, 0)
    
    # Find contours to detect squares
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Placeholder list to store positions of green squares
    positions = []
    
    for contour in contours:
        # Compute the center of the contour
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            
            # Ignore the center element (based on contour area, adjust the threshold as needed)
            if cv2.contourArea(contour) < 5000:  # 5000 is an example threshold, adjust as necessary
                positions.append((cX, cY))
                
                # Draw a circle at the detected position on the overlay
                cv2.circle(overlay, (cX, cY), 5, (0, 0, 255), -1)  # Drawing in red for visibility

    # Display the overlay
    cv2.imshow("Overlay", overlay)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    print(type(positions))
    print(positions)
    return positions
    

detect_green_squares_and_overlay(crossword_path)



<class 'list'>
[(308, 703), (85, 591), (587, 422), (252, 254), (29, 29)]


[(308, 703), (85, 591), (587, 422), (252, 254), (29, 29)]

This bit attempts to get a text representation of the crossword grid

In [24]:
import pytesseract
import cv2
import numpy as np

# Load the image and parameters
grid_img = cv2.imread('grid.png', cv2.IMREAD_COLOR)
# Crop 3 pixels off every side
grid_img = grid_img[3:-3, 3:-3]
grid_img = cv2.cvtColor(grid_img, cv2.COLOR_BGR2RGB)
num_rows = 13
num_cols = 13
cell_height = grid_img.shape[0] // num_rows
cell_width = grid_img.shape[1] // num_cols
positions = [(308, 703), (85, 591), (587, 422), (252, 254), (29, 29)]

# Extract cells from the image
cells = [[grid_img[i*cell_height:(i+1)*cell_height, j*cell_width:(j+1)*cell_width] 
          for j in range(num_cols)] for i in range(num_rows)]

# Functions
def is_black(cell):
    avg_color = np.mean(cell)
    return avg_color < 50

def potential_number_cell(cell):
    gray = cv2.cvtColor(cell, cv2.COLOR_RGB2GRAY)
    thresholded = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY_INV)[1]
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) > 50:
            return True
    return False

def extract_number(cell):
    resized_cell = cv2.resize(cell, (cell.shape[1] * 3, cell.shape[0] * 3), interpolation=cv2.INTER_LINEAR)
    gray = cv2.cvtColor(resized_cell, cv2.COLOR_RGB2GRAY)
    thresholded = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY_INV)[1]
    kernel = np.ones((3, 3), np.uint8)
    dilated = cv2.dilate(thresholded, kernel, iterations=1)
    number = pytesseract.image_to_string(dilated, config='--psm 6').strip()
    return number if number.isdigit() else None

def crop_cell_border(cell, percentage=8):  # Changed to 4%
    h, w, _ = cell.shape
    vertical_crop = int(h * percentage / 100)
    horizontal_crop = int(w * percentage / 100)
    return cell[vertical_crop:-vertical_crop, horizontal_crop:-horizontal_crop]

# Refine potential number-containing cells
refined_potential_number_cells = []
for i in range(num_rows):
    for j in range(num_cols):
        cell = cells[i][j]
        if 5 <= i <= 7 and 5 <= j <= 7 or is_black(cell):
            continue
        if potential_number_cell(cell):
            refined_potential_number_cells.append((i, j, cell))
import pytesseract
import cv2
import numpy as np

# Load the image and parameters
grid_img = cv2.imread('grid.png', cv2.IMREAD_COLOR)
# Crop 3 pixels off every side
grid_img = grid_img[3:-3, 3:-3]
grid_img = cv2.cvtColor(grid_img, cv2.COLOR_BGR2RGB)
num_rows = 13
num_cols = 13
cell_height = grid_img.shape[0] // num_rows
cell_width = grid_img.shape[1] // num_cols
positions = [(308, 703), (85, 591), (587, 422), (252, 254), (29, 29)]

# Extract cells from the image
cells = [[grid_img[i*cell_height:(i+1)*cell_height, j*cell_width:(j+1)*cell_width] 
          for j in range(num_cols)] for i in range(num_rows)]

# Functions
def is_black(cell):
    avg_color = np.mean(cell)
    return avg_color < 50

def extract_number_from_corner(cell):
    # Crop the top-left portion (considering it's about 1/3rd of the cell size)
    corner = cell[:cell_height//3, :cell_width//3]
    resized_corner = cv2.resize(corner, (corner.shape[1] * 3, corner.shape[0] * 3), interpolation=cv2.INTER_LINEAR)
    gray = cv2.cvtColor(resized_corner, cv2.COLOR_RGB2GRAY)
    thresholded = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY_INV)[1]
    number = pytesseract.image_to_string(thresholded, config='--psm 6').strip()
    return number if number.isdigit() else None

# Extract numbers from the top-left corner of the cell
number_data = {}
for i in range(num_rows):
    for j in range(num_cols):
        cell = cells[i][j]
        number = extract_number_from_corner(cell)
        if number:
            number_data[(i, j)] = number

# Create the final textual representation
crossword_textual_final = []
for i in range(num_rows):
    row_representation = []
    for j in range(num_cols):
        cell = cells[i][j]
        cell_center = (j * cell_width + cell_width // 2, i * cell_height + cell_height // 2)
        if 5 <= i <= 7 and 5 <= j <= 7:
            row_representation.append('X')
        elif cell_center in positions:
            if (i, j) in number_data:
                row_representation.append(f"({number_data[(i, j)]}G)")
            else:
                row_representation.append('G')
        elif is_black(cell):
            row_representation.append('X')
        elif (i, j) in number_data:
            row_representation.append(f"({number_data[(i, j)]})")
        else:
            row_representation.append('O')
    crossword_textual_final.append("".join(row_representation))

crossword_textual_representation_final = "\n".join(crossword_textual_final)
print(crossword_textual_representation_final)


# Extract numbers with 4% border cropping
number_data_cropped_4 = {}  # Changed variable name for clarity
for i, j, cell in refined_potential_number_cells:
    cropped_cell = crop_cell_border(cell)
    number = extract_number(cropped_cell)
    if number:
        number_data_cropped_4[(i, j)] = number

# Create the final textual representation
crossword_textual_final = []
for i in range(num_rows):
    row_representation = []
    for j in range(num_cols):
        cell = cells[i][j]
        cell_center = (j * cell_width + cell_width // 2, i * cell_height + cell_height // 2)
        if 5 <= i <= 7 and 5 <= j <= 7:
            row_representation.append('X')
        elif cell_center in positions:
            if (i, j) in number_data_cropped_4:
                row_representation.append(f"({number_data_cropped_4[(i, j)]}G)")
            else:
                row_representation.append('G')
        elif is_black(cell):
            row_representation.append('X')
        elif (i, j) in number_data_cropped_4:
            row_representation.append(f"({number_data_cropped_4[(i, j)]})")
        else:
            row_representation.append('O')
    crossword_textual_final.append("".join(row_representation))

crossword_textual_representation_final = "\n".join(crossword_textual_final)
print(crossword_textual_representation_final)


(1)O(12)OO(13)X(4)OOOOO
OXOXX(17)OOXOXOO
(8)OOOOOXOXOOOO
OXXOXOXOXOXOO
XOXOOOOO(1)OXOO
(1)OOOOXXXOOOOO
XOXXOXXXOXXOO
OOO(4)OXXX(14)OOOO
XOXOOOOOOOXOO
XOXOXOXOXOXXO
OOOOXOX(9)OOOOO
XOXOXOOOXXOOO
OOOOOOXOOOOOO


In [ ]:
# Example Usage
image_path = 'path_to_crossword_image.png'
across_hints, down_hints = extract_hints_from_image(image_path)
print("Across Hints:\n", across_hints)
print("\nDown Hints:\n", down_hints)

green_square_positions = detect_green_squares_and_overlay(crossword_path)
print("\nPositions of green squares:", green_square_positions)


The final bit attempts to feed the summarized text data above into GPT-4 and prompt it to resolve the puzzle and return the characters that correspond to the green letters